In [103]:
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [269]:
hotel = pd.read_csv('hotel.csv')
hotel.drop(hotel.columns[0],axis=1, inplace=True)

In [270]:
X = hotel.copy()
y = hotel.pop('is_canceled')

In [271]:
X['arrival_date_month'] = \
                    X['arrival_date_month'].map({'January':1, 'February': 2, 'March':3,
         'April':4, 'May':5, 'June':6, 'July':7,
         'August':8, 'September':9, 'October':10,
         'November':11, 'December':12})

In [272]:
feature_num = ["lead_time", "arrival_date_week_number",
    "arrival_date_day_of_month", "stays_in_weekend_nights",
    "stays_in_week_nights", "adults", "children", "babies",
    "is_repeated_guest", "previous_cancellations",
    "previous_bookings_not_canceled", "required_car_parking_spaces",
    "total_of_special_requests", "adr"]


feature_cat = ["hotel", "arrival_date_month", "meal",
    "market_segment", "distribution_channel",
    "reserved_room_type", "deposit_type",]

In [273]:
transformer_num = make_pipeline(
    SimpleImputer(strategy='constant', fill_value=0),
    StandardScaler(),
)

In [274]:
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value='NA'),
    OneHotEncoder(handle_unknown='ignore'),

)

In [275]:
preprocessor = make_column_transformer(
                (transformer_num,feature_num),
                (transformer_cat,feature_cat),
)

In [276]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.70)

In [277]:
X_train = preprocessor.fit_transform(X_train)

In [278]:
X_valid = preprocessor.fit_transform(X_valid)

In [279]:
input_shape = [X_train.shape[1]]

In [280]:
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(83573, 59) (35817, 58) (83573,) (35817,)


In [281]:
model = keras.Sequential([
        # First layer is Batch Normalization
        layers.BatchNormalization(input_shape=input_shape),
        # Second layer is Dense 
        layers.Dense(units=512,activation='relu'),
        # Third layer of Batch Normalization
        layers.BatchNormalization(),
        # Fourth layer of droput
        layers.Dropout(0.3),
        # Fifth layer of dense
        layers.Dense(units=512, activation='relu'),
        # Sixth layer of Batch Normalization
        layers.BatchNormalization(),
        # Seventh layer of dropout
        layers.Dropout(0.3),
        # Final layer
        layers.Dense(units=1, activation='sigmoid'),
    
])

In [282]:
METRICS = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    
    
]

model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=METRICS,

)

early_stopping = keras.callbacks.EarlyStopping(
                        patience=10,
                        min_delta=0.001,
                        restore_best_weights=True,

)

In [283]:
history = model.fit(
                    X_train,y_train,
                    validation_data=(X_valid,y_valid),
                    batch_size=512,
                    epochs=200,
                    verbose=0,
                    callbacks=[early_stopping],


)

ValueError: in user code:

    /Users/bhupendrasinhthakre/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /Users/bhupendrasinhthakre/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/bhupendrasinhthakre/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/bhupendrasinhthakre/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/bhupendrasinhthakre/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwarg

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[0:,['loss','val_loss']])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.show()

In [ ]:
plt.plot(history_df.loc[0:,['accuracy','val_accuracy']])

In [ ]:
history_df.head()